In [5]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 133.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
pip install -U bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.46.0
    Uninstalling bitsandbytes-0.46.0:
      Successfully uninstalled bitsandbytes-0.46.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0


Imports

In [19]:
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import csv
import gradio as gr

Login to hugging Face

In [2]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Import Open-source model from HuggingFace

In [28]:
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct"
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [29]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

Generate Dataset Function

In [36]:
def generate_dataset(model, num_samples, product_info):

    # Step 1: Specify instructions for the model with a system message
    system_message = """
                    You are a helpful assistant that generates synthetic product reviews along with a label ("positive" or "negative"). Can't have neutral reviews.
                    Return exactly N examples (specify count via a parameter) that cover a variety of styles and lengths.
                    Format the output as a downloadable JSON array of objects:

                    [
                      {
                        "review_id": "...",
                        "review_text": "...",
                        "sentiment_label": "positive" or "negative"
                      },
                      ...
                    ]

                    Ensure:
                    - A balanced mix of positive and negative reviews.
                    - Each review is clearly synthetic (do not mimic real reviews).
                    - Reviews are realistic, diverse in tone and length.

                    Here are some examples:

                    {
                      "review_id": "0001",
                      "review_text": "I absolutely love this phone case — it’s durable, fits perfectly, and the design is sleek!",
                      "sentiment_label": "positive"
                    }

                    {
                      "review_id": "0002",
                      "review_text": "The speaker stopped working after a week; customer service was completely unhelpful.",
                      "sentiment_label": "negative"
                    }

                    {
                      "review_id": "0003",
                      "review_text": "Battery life is decent, but charging is slower than expected.",
                      "sentiment_label": "negative"
                    }
                    """


    user_prompt = f"Please generate my dataset for {product_info} with {num_samples} samples."

    # Step 2: Set the tokenizer for the model
    tokenizer = AutoTokenizer.from_pretrained(LLAMA)
    tokenizer.pad_token = tokenizer.eos_token
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_prompt}]

    #Step 3: Load the model
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
    streamer = TextStreamer(tokenizer)
    model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=quant_config)

    #Step 4: Generate output
    outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

    #Step 5: Decode and return
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



Create Gradio Interface

In [37]:
model_choices = [
    ("LLaMA", LLAMA),
    ("Phi‑3 mini", PHI3),
    ("Gemma‑2", GEMMA2),
    ("Qwen‑2", QWEN2),
    ("Mixtral‑8x7B", MIXTRAL),
]

ui = gr.Interface(
    fn=generate_dataset,
    inputs=[
        gr.Dropdown(model_choices, label="Model", value=model_choices[0][1]),
        gr.Number(label="Number of Samples"),
        gr.Textbox(label="Product Info")
    ],
    outputs=gr.Textbox(label="Generated Dataset")
)

In [38]:
ui.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff75d081f8b668da72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
